Evan Callia, Kyle Kusuda, Zen, Denny, Allie

In [1]:
#Importing appropriate modules
import sys
sys.path.append('C:\Python27\ArcGIS10.3\Lib\site-packages')
import pg
from pg import DB
import csv

ImportError: DLL load failed: The specified module could not be found.

In [2]:
#connecting to databse
db = DB(dbname = 'geog465_evcallia', host = 'geog-db2.geog.uw.edu', port = 5432, user = 'evcallia', passwd = 'evcallia')

Creating tables in databse

Constraints are left out becasue they will have to be removed when populationg the tables. They will be added later

In [ ]:
#Permits
db.query('CREATE TABLE Permits (PermitNumber int PRIMARY KEY, Description varchar(500), Value money, ApplicantName varchar(75), ApplicationDate varchar(25), IssueDate varchar(25), FinalDate varchar(25), ExpirationDate varchar(25), Contractor varchar(75), URL varchar(500), MasterNumber int, Statuses_id int, Permit_Types_id int, Categories_id int, Work_Types_id int, Location_id int, Actions_id int)')

In [ ]:
#Actions
db.query('CREATE TABLE Actions (id int PRIMARY KEY, ActionType varchar(75))')

In [ ]:
#Categories
db.query('CREATE TABLE Categories (id int PRIMARY KEY, Category varchar(25)')

In [ ]:
#Location
db.query('CREATE TABLE Location (id int PRIMARY KEY, Address varchar(75), Latitude varchar(20), Longitude varchar(20), Location varchar(35))')

In [ ]:
#PermitTypes
db.query('CREATE TABLE PermitTypes (id int PRIMARY KEY, PermitType varchar(25)')

In [ ]:
#Statuses
db.query('CREATE TABLE Statuses (id int PRIMARY KEY, Status varchar(50))') 

In [ ]:
#WorkTypes
db.query('CREATE TABLE WorkTypes (id int PRIMARY KEY, WorkType varchar(25))')

In [ ]:
#LinkTable 
db.query('CREATE TABLE LinkTable (PermitNumber int PRIMARY KEY, ActionType varchar(75), Category varchar(25), Address varchar(75), PermitType varchar(25), Status varchar(50), WorkType varchar(25))')

Populating Tables

In [ ]:
#make lsit of csvs and tables to interate through
csv_list = ['permits.csv', 'actions.csv', 'categories.csv', 'permit_types.csv', 'location.csv', 'statuses.csv', 'work_types.csv', 'Building_Permits_Current.csv']
table_list = ['Permits', 'Actions', 'Categries', 'PermitTypes', 'Location', 'Statuses', 'WorkTypes', 'LinkTable']
index_counter = 0

for csv in csv_list:
    f = open('enter/pathway/here/' + csv)
    reader = csv.reader(f, delimiter = ',')
    all_rows_list = []
    next(reader, None) #skip header row
    for row in reader:
        if csv != 'Building_Permits_Current.csv':   #contidional statement to see if LinkTable
            row_list = []
            for index in row:
                row_list.append(index)
            all_rows_list.append(row_list)
        else:
            #this populates the LinkTable which is a special case and indexes must be hard coded
            all_rows_list.append(row[0], row[5], row[4], row[2], row[1], row[13], row[6])
    f.close()
    db.inserttable(table_list[index_counter], all_rows_list)
    index_counter += 1

Populating foreign keys

In [ ]:
#FK field Statuses_id
db.query('UPDATE Permits SET Statuses_id = Statuses.id FROM Statuses, linkTable WHERE linkTable.Status = Statuses.Status AND linkTable.PermitNumber = Permits.PermitNumber')

In [ ]:
#FK field Permit_Types_id
db.query('UPDATE Permits SET Permit_Types_id = PermitTypes.id FROM PermitTypes, linkTable WHERE linkTable.PermitType = PermitTypes.PermitType AND linkTable.PermitNumber = Permits.PermitNumber')

In [ ]:
#FK field Categories_id
db.query('UPDATE Permits SET Categories_id = Categories.id FROM Categories, linkTable WHERE linkTable.Category = Categories.Category AND linkTable.PermitNumber = Permits.PermitNumber')

In [ ]:
#FK field Work_Types_id
db.query('UPDATE Permits SET Work_Types_id = WorkTypes.id FROM WorkTypes, linkTable WHERE linkTable.WorkType = WorkTypes.WorkType AND linkTable.PermitNumber = Permits.PermitNumber')

In [ ]:
#FK field Location_id
db.query('UPDATE Permits SET Location_id = Location.id FROM Location, linkTable WHERE linkTable.Address = Location.Address AND linkTable.PermitNumber = Permits.PermitNumber')

In [ ]:
#FK field Actions_id
db.query('UPDATE Permits SET Actions_id = Actions.id FROM Actions, linkTable WHERE linkTable.ActionType = Actions.ActionType AND linkTable.PermitNumber = Permits.PermitNumber')

In [ ]:
#add constriants back to tables